<a href="https://colab.research.google.com/github/Loga19818eeanvesh/CNN_Models/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
print(tf.__version__)

2.8.0


In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,ReLU,BatchNormalization,MaxPool2D,GlobalAveragePooling2D,Dense


In [5]:
class ResidualBlock(Model):
  def __init__(self,kernel_size,noof_filters,input_depth,stride=(1,1)):
    super().__init__()
    self.conv1 = Conv2D(filters=noof_filters,kernel_size=kernel_size,strides=stride)
    self.conv2 = Conv2D(filters=noof_filters,kernel_size=kernel_size,strides=stride)
    self.relu = ReLU()
    self.BatchNorm1 = BatchNormalization()
    self.BatchNorm2 = BatchNormalization()
    if input_depth!=noof_filters:
      self.conv3(filters=noof_filters,kernel_size=1)

  def __call__(self,inputs):
    x = self.conv1(inputs);
    x = self.BatchNorm1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.BatchNorm2(x)
    if(self.conv3!=None):
      inputs = self.conv3(inputs)
    x = self.relu(inputs+x)
    return x

In [13]:
class ResNetBlock(Model):
  def __init__(self,kernel_size,noof_filters,input_depth,noof_blocks,stride=(1,1)):
    super().__init__()
    self.noof_blocks = noof_blocks
    ResidualBlocks = {}
    ResidualBlocks["ResidualBlock0"] = ResidualBlock(kernel_size=kernel_size,noof_filters=noof_filters,input_depth=input_depth,stride=stride)
    for i in range(noof_blocks):
      if i==0 :
        continue
      ResidualBlocks["ResidualBlock"+str(i)] = ResidualBlock(kernel_size=kernel_size,noof_filters=noof_filters,input_depth=noof_filters,stride=stride)
    self.Blocks = ResidualBlocks

  def __call__(self,inputs):
    x = self.Blocks["ResidualBlock0"](inputs)
    for i in range(self.noof_blocks):
      if i==0:
        continue
      x = self.Blocks["ResidualBlock"+str(i)](x)

    return x


In [14]:
class ResNet(Model):
  def __init__(self,input_shape,input_filters,kernel_shape,outputs):
    super().__init__()
    self.conv1 = Conv2D(input_shape=input_shape,filters=input_filters)
    self.BatchNorm = BatchNormalization()
    self.maxpool = MaxPool2D()
    self.relu = ReLU()
    self.ResNetBlock1 = ResNetBlock(kernel_size=kernel_shape,noof_filters=64,input_depth=input_filters,noof_blocks=3)
    self.ResNetBlock2 = ResNetBlock(kernel_size=kernel_shape,noof_filters=128,input_depth=64,noof_blocks=4)
    self.ResNetBlock3 = ResNetBlock(kernel_size=kernel_shape,noof_filters=256,input_depth=128,noof_blocks=6)
    self.ResNetBlock4 = ResNetBlock(kernel_size=kernel_shape,noof_filters=512,input_depth=256,noof_blocks=3)
    self.GlobalAvgPool = GlobalAveragePooling2D()
    self.classifier = Dense(units=outputs,activation=tf.keras.activations.softmax)

  def __call__(self,inputs):
    x = self.conv1(inputs)
    x = self.BatchNorm(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.ResNetBlock1(x)
    x = self.ResNetBlock2(x)
    x = self.ResNetBlock3(x)
    x = self.ResNetBlock4(x)
    x = self.GlobalAvgPool(x)
    x = self.classifier(x)
    return x